This notebook gives a simple example of how to use the ShaderMaterial to write custom shaders from the Python side. For further information about the shaders, consult the three.js docs.

In [ ]:
from pythreejs import *
import ipywidgets as widgets
from IPython.display import display

In [ ]:
ShaderMaterial?

In [ ]:
vertex_shader = """
uniform float time;
uniform vec2 resolution;

void main() {
    vec3 pos = vec3(position.x + time * resolution.x, position.y + time * resolution.y, position.z);
    gl_Position = projectionMatrix * modelViewMatrix * vec4( pos, 1.0 );
}
"""

In [ ]:
fragment_shader = """
uniform vec3 user_color;

void main() {
    #ifdef FIX_COLOR
    gl_FragColor = vec4(0.5, 0, 0, 1.0);
    #else
    gl_FragColor = vec4(user_color, 1.0);
    #endif
}
"""

In [ ]:
material = ShaderMaterial(
    uniforms=dict(
        time=dict(value=0.0),
        resolution=dict(value=(1, 1)),
        user_color=dict(value='green'),
        **UniformsLib['common']
    ),
    defines=dict(
        FIX_COLOR=1,
    ),
    vertexShader=vertex_shader,
    fragmentShader=fragment_shader,
)

In [ ]:
material

In [ ]:
# Disable fixed color from defines:
material.defines = dict()
material.needsUpdate = True

In [ ]:
# Uniform editors:
picker = widgets.ColorPicker(value=material.uniforms['user_color']['value'])
time_slider = widgets.FloatSlider(value=material.uniforms['time']['value'], min=-15, max=15)

# Interactive code:
def update_uniforms(change):
    uniforms = dict(**material.uniforms)
    uniforms.update(
        time=dict(value=time_slider.value),
        user_color=dict(value=picker.value),
    )
    material.uniforms = uniforms
    material.needsUpdate = True

picker.observe(update_uniforms)
time_slider.observe(update_uniforms)

# Present "dashboard" in VBox:
widgets.VBox([
    Preview(material),
    picker,
    time_slider,
])